## Задача размещения баннеров.
### Постановка задачи.

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat. Открыв его, увидим следующую структуру: 

id | user_id | venue_id | latitude | longitude | created_at

---------+---------+----------+-------------------+-------------------+---------------------

984301 | 2041916 | 5222 | | | 2012-04-21 17:39:01

984222 | 15824 | 5222 | 38.8951118 | -77.0363658 | 2012-04-21 17:43:47

984315 | 1764391 | 5222 | | | 2012-04-21 17:37:18

984234 | 44652 | 5222 | 33.800745 | -84.41052 | 2012-04-21 17:43:43

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас.

С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание:на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

### Решение задачи:

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [2]:
# загрузим таблицу с данными, явно задав тип данных для нужных нам столбцов latitude и longitude. 
# Это позволит нам найти все неккоректно заполненные значения.
data = pd.read_csv('C:\\Users\\andre\\Downloads\\fsq\\checkins.csv', sep='|', 
                   skipinitialspace=True, dtype={'latitude': float, 'longitude': float})
data.columns = data.columns.str.replace(' ', '')

In [3]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984301,2041916.0,5222.0,NaN,NaN,2012-04-21 17:39:01
1,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
2,984315,1764391.0,5222.0,NaN,NaN,2012-04-21 17:37:18
3,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
4,984249,2146840.0,5222.0,NaN,NaN,2012-04-21 17:42:58


In [4]:
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

In [5]:
# чтобы ускорить работу алгоритма, возьмем 100 тысяч случайных объектов из выборки и удалим лишние столбцы 
data = data.sample(100000)
data = data.drop(['created_at', 'id', 'user_id', 'venue_id'], axis=1)

In [6]:
from sklearn.cluster import MeanShift

In [7]:
clusterizator = MeanShift(bandwidth=0.1)

In [8]:
%%time
clusters = clusterizator.fit_predict(data)
clusters

Wall time: 2min 45s


array([68, 63,  0, ...,  7,  9,  0], dtype=int64)

In [9]:
# добавим столбец с результатами кластеризации в исходную таблицу, чтобы было удобнее обрабатывать данные в дальнейшем.
data['clusters'] = clusters

In [10]:
# найдет максимальный номер кластера
max_cluster = data.clusters.max()

In [11]:
# и удалим все кластеры, которые содержат до 15 элементов включительно

for x in range(max_cluster):
    if data[(data['clusters'] ==x)].count()[2] < 16:
        indexes = data[(data['clusters'] ==x)].index
        data.drop(indexes, inplace=True)


In [19]:
# извлечем координаты центров каждого кластера
centers = clusterizator.cluster_centers_
centers.shape

(3651, 2)

In [13]:
# и сохраним их в файл, чтобы визуализировать кластеры с помощью готового инструмента
dataset = pd.DataFrame({'Column1': centers[:, 0], 'Column2': centers[:, 1]})
dataset.to_csv('centers.csv', index=False)

In [14]:
# сохраним координаты офисов, которые даны по условию задачи
offices = np.array([
    [33.751277, -118.188740],
    [25.867736, -80.324116],
    [51.503016, -0.075479],
    [52.378894, 4.885084],
    [39.366487, 117.036146],
    [-33.868457, 151.205134]])

In [15]:
from scipy.spatial.distance import euclidean

In [16]:
# найдем расстояния от центров кластеров до каждого из офисов 
# и сохраним их в переменную distances вместе с соответствующими координатами: 

distances = []
for x in centers:
    k=0
    while k < len(offices):
        dist = euclidean(x, offices[k])
        distances.append((dist, x))
        k +=1
       
        

In [23]:
# отсортируем расстояния от меньшего к большему и отберем 20 ближайших к офисам кластеров
closest_centers = sorted(distances)[:20]
closest_centers

[(0.0014416052249948213, array([-33.86713969, 151.2057196 ])),
 (0.010167671838629787, array([52.3714335 ,  4.89199215])),
 (0.05172314498122836, array([51.5022206 , -0.12719603])),
 (0.07436664675266587, array([  33.81163755, -118.14530037])),
 (0.12127166897598157, array([ 25.90152941, -80.20764788])),
 (0.13244949716601173, array([5.14056832e+01, 1.43497000e-02])),
 (0.1342922486772476, array([ 25.78755099, -80.21639055])),
 (0.14885159278721202, array([52.505    ,  4.9641667])),
 (0.15329917431339365, array([51.35659493, -0.12088283])),
 (0.15527519749498295, array([52.28547395,  4.76105545])),
 (0.17091152470263668, array([52.3080507,  5.0406217])),
 (0.1719736731634021, array([-33.90847715, 151.0378817 ])),
 (0.18301232408573287, array([ 26.00364976, -80.20155597])),
 (0.18869933332347014, array([  33.8931542 , -118.06432819])),
 (0.21577445361255318, array([  33.87024537, -118.36875428])),
 (0.24453231546975557, array([52.3802778,  4.6405556])),
 (0.26681671356123293, array([  3

In [70]:
# напечатаем координаты ближайшего центра

print('Coordinates of the closest cluster center to Carnival Cruise Line agency: {}'
      .format(', '.join(list(map(str, closest_centers[0][1])))))

Coordinates of the closest cluster center to Carnival Cruise Line agency: -33.86713969444442, 151.20571960370384
